# **Parking Space**

Bu projede otopark kamera kaydı görüntülerinden kaç aracın park ettiğinin ve kaç alanın boş olduğunun anlık olarak tespiti gerçekleştirilecektir.

Parking Space Picker kısmında otoparktaki boş ve dolu park alanları işaretlenecektir ve bu alanların boş mu dolu mu olduğuna algoritma karar verecektir.

Parking Space Counter ise algoritmanın çalışacağı kısımdır.

**Parking Space Picker**

First frame adından da anlaşılacağı üzere videonun ilk frame'idir.

First frame kullanılarak park alanları işaretlenecektir.

İşaretlenen alanların dolu mu boş mu olduğunu algoritma değerlendirecektir.

In [ ]:
import cv2
import pickle #pickle kütüphanesi ile park poziyonları kaydedilir, görüntünün üzerine park pozisyonları otomatik bir şekilde işlenecektir.

In [ ]:
width = 70
height = 20

In [ ]:
#read first frame
try:
    with open("Car_Park_Positions","rb") as f:
            position_list = pickle.load(f)
except:
    position_list = []

In [ ]:
def mouse_click(events, x, y, flags, params):
    
    if events == cv2.EVENT_LBUTTONDOWN:
        position_list.append((x,y))
        
    if events == cv2.EVENT_RBUTTONDOWN:
        
        for i, positions in enumerate(position_list):
            x1, y1 = positions
            
            if x1 < x < x1 + width and y1 < y < y1 + height:
                position_list.pop(i)
                
    with open("Car_Park_Positions","wb") as f:
        pickle.dump(position_list, f)
    #işaretlenen alanlar kaydedilir.

mouse_click fonksiyonu kullanılarak görüntüye tıklandığında ilgili yere dikdörtgen atanır.

Dikdörtgen içerisindeki park alanının dolu mu boş mu olduğu anlaşılmaya çalışılacaktır.

Sol click kutu eklemek, sağ click yanlış bir yere kutu eklendiğinde silmek için kullanılır.

In [ ]:
while True:
    
    #image read
    frame = cv2.imread(r"C:\Users\ilhan\Desktop\first_frame.png")
    
    scale_percent = 220 # percent of original size
    w = int(frame.shape[1] * scale_percent / 100)
    h = int(frame.shape[0] * scale_percent / 100)
    dim = (w, h)
 
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
    #print("Position List: ",position_list)
    #mouse'un sol tuşuna basıldığında, o noktanın x,y koordinatları yazdırılır. 
    
    for positions in position_list:
        cv2.rectangle(frame, positions, (positions[0] + width, positions[1] + height), (255,0,0), 2)
    
    cv2.imshow("First Frame", frame)
    cv2.setMouseCallback("First Frame", mouse_click)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
cv2.destroyAllWindows()

**Parking Space Counter**

In [ ]:
import cv2
import pickle
import numpy as np

In [ ]:
def check_park_space(image):
    space_counter = 0
    
    for position in position_list:
        x, y = position
        
        crop_image = image[y: y + height, x:x + width]
        count = cv2.countNonZero(crop_image)
        
        #print("count: ", count)
        #dolu ve boş olan park alanlarının count değerine bakılarak threshold belirlenir.
        
        if count < 150:
            color = (0, 255, 0)
            space_counter += 1
        else:
            color = (0, 0, 255)
    
        cv2.rectangle(frame, position, (position[0] + width, position[1] + height), color, 2) 
        cv2.putText(frame, str(count), (x, y + height - 2), cv2.FONT_HERSHEY_PLAIN, 1,color,1)
        
    cv2.putText(frame, f"Free Space Ratio: {space_counter}/{len(position_list)}", (15,24), cv2.FONT_HERSHEY_PLAIN, 2,(0,255,255),4)

In [ ]:
#video capture
capture = cv2.VideoCapture(r"C:\Users\ilhan\Desktop\video.mp4")

In [ ]:
width = 70
height = 20

In [ ]:
with open("Car_Park_Positions","rb") as f:
            position_list = pickle.load(f)

In [ ]:
while True:
    
    ret, frame = capture.read()
    
    if ret == True:
        
        scale_percent = 220 # percent of original size
        w = int(frame.shape[1] * scale_percent / 100)
        h = int(frame.shape[0] * scale_percent / 100)
        dim = (w, h)

        # resize image
        frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        
        #converting bgr to grayscale format
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #gaussian blur ile noise giderilir ve detaylar azaltılır.
        blur_frame = cv2.GaussianBlur(gray_frame, (3,3), 1)
        
        #thresholding işlemi ile gereksiz detaylar ortadan kaldırılıp ana hatlar vurgulanır.
        #bu projede ana hatlar dolu olan park alanlarıdır.
        threshold_frame = cv2.adaptiveThreshold(blur_frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 16)
        
        #threshold yapıldıktan bazı boş park alanlarının çizgileri giderilmemiştir.
        #median blur ile bu pikseller arındırılarak detaylar ortadan kaldırılır.
        median_frame = cv2.medianBlur(threshold_frame, 5)
        
        #dilation işlemi ile beyaz yerler kalınlaştırılır.
        #threshold işlemi sonrası park alanları beyaz olduğundan bu yöntem tercih edilmiştir.
        dilate_frame = cv2.dilate(median_frame, np.ones((3,3)), iterations = 1)
        
        check_park_space(dilate_frame)

        cv2.imshow("Parking Space Counter", frame)
        #cv2.imshow("Gray Scale", gray_frame)
        #cv2.imshow("Gaussian Blur", blur_frame)
        #cv2.imshow("Thresholding", threshold_frame)
        #cv2.imshow("Median Blur", median_frame)
        #cv2.imshow("Dilation", dilate_frame)

        if cv2.waitKey(200) & 0xFF == ord('q'):
                break
                
    else: break
            
cv2.destroyAllWindows()